# Best Reciprocal Hit 

As explained later in this notebook, we need ALL the result of the blast between each species and we also need ALL the proteomes.

## Import

In [2]:
%config Completer.use_jedi = False
import os
import numpy as np
import pandas as pd
from Bio import SeqIO

ALL_filename = []
for i in os.listdir():
    if i[-3:] == '.m8':
        ALL_filename.append(i)

This code need to be in the directory with the blast results, or need to change to path of the os.listdir() or even os.chdir() to the directory where those results are

## Functions

In [25]:
def split_filename(filename):
    name1 , name2 = filename.split('_v_')
    name2 = name2[:-3]
    return name1 , name2

def make_filename(nom1,nom2):
    return nom1+'_v_'+nom2+'.m8'


def split_name_gene(filename):
    name , gene = filename.split('_w_')    
    return name , gene

def make_ID(name,gene):
    return name+'_w_'+gene


def reverse_name(filename):
    name1 , name2 = filename.split('_v_')
    name2 = name2[:-3]
    name1 = name1+'.m8'
    return name2+'_v_'+name1

def is_in(name ,filename):
    name1 , name2 = split_filename(filename)
    if name == name1 or name == name2:
        return True
    else:
        return False

def is_in_name1(name ,filename):
    name1 , name2 = split_filename(filename)
    if name == name1 :
        return True
    else:
        return False
    
def is_in_name2(name ,filename):
    name1 , name2 = split_filename(filename)
    if name == name2 :
        return True
    else:
        return False
    
def colname_to_proteome(C):
    return C+'.fasta.transdecoder.pep'
    
    
def make_dics(file):
    dic={}
    flux= open(file,'r')
    name1 , name2 = split_filename(file)
    for ligne in flux:
        ligne = ligne.split()
        if float(ligne[10]) <= 1e-3:
            dic[name1+'_w_'+ligne[0]] = name2+'_w_'+ligne[1]
    flux.close()
    return dic

## Extrating all the files

Extracting all the files from diamond operation (here we have 506 files), we are creating a dictionary of dictionary with the structure :  big_dictionary[filename][protein]=[best_hit]


In [26]:
#long operation ~3,5min
BIG_DIC = {}
a=0
inter=[i for i in range(0,len(ALL_filename),50)]
for i in ALL_filename:
    
    BIG_DIC[i] = make_dics(i)
    if a in inter:
        print(a,"/",len(ALL_filename))
    a+=1

0 / 506
50 / 506
100 / 506
150 / 506
200 / 506
250 / 506
300 / 506
350 / 506
400 / 506
450 / 506
500 / 506


In [28]:
print('number of dictionary : ',len(BIG_DIC))

number of dictionary :  506


### extracting all species names
Not in the same loop juste for the time management

In [29]:
liste_name=[]
for i in ALL_filename:
    liste_name.append(split_filename(i)[0])
    liste_name.append(split_filename(i)[1])
liste_name=list(np.unique(liste_name))

## Function in order to find BRH

In [30]:
def is_BRH (name1 , name2 , ID , dic):
    filename1 = make_filename(name1,name2)
    if ID in dic[filename1].keys() and dic[filename1][ID] in dic[reverse_name(filename1)] :
        if ID == dic[reverse_name(filename1)][dic[filename1][ID]]:
            return True
        
        else:
            return False
    else:
        return False
    
    
def BRH (ID , list_spec , dic):
    liste_BRH=[ID]
    for i in list_spec:
        ID = liste_BRH[-1]
        for j in list_spec:
            if j != i :
                filename = make_filename(i,j)
                reversename = reverse_name(filename)
                if is_BRH(i,j,ID ,dic) == True:
                    if ID not in liste_BRH:
                        liste_BRH.append(ID)
                    if dic[filename][ID] not in liste_BRH:
                        liste_BRH.append(dic[filename][ID])
                else:
                    break
    return liste_BRH

## Finding BRH

Running it just for the first one

In [31]:
liste_BEST=[]
a=0
for i in BIG_DIC.keys():
    print(i)
    for query in BIG_DIC[i]:
        a+=1
        liste_BEST.append(BRH(query,liste_name,BIG_DIC))
    break
    print(len(liste_BEST))
    

Penaeus-aztecus_10Hepatopancreas_outGroup_SRR3711052_Trinity_v_Amphibalanus-amphitrite_larva_outGroup_SRR1525781_Trinity.m8


### Taking only the one equal to the number of species

In [32]:
liste_sort=[]
print('avant le tri : ',len(liste_BEST))
for i in liste_BEST:
    if len(i) == 23 : 
        liste_sort.append(i)

print('après le tri : ',len(liste_sort))



avant le tri :  23896
après le tri :  190


### Making sure they have one protein for each species

In [33]:
print("before removing none satisfying : ",len(liste_sort))
a=0
liste_final=[]
for i in liste_sort:
    #print(i)
    list_tmp=[]
    for ID in i:   
        name , gene = split_name_gene(ID)
        list_tmp.append(name)

    if sorted(list_tmp) == sorted(liste_name):
        liste_final.append(i)
    a+=1
print("after removing none satisfying : ",len(liste_final))


before removing none satisfying :  190
after removing none satisfying :  176


### making the .csv to save them

In [34]:
df_final=pd.DataFrame(np.zeros((len(liste_final),len(liste_name))),columns=liste_name,dtype=str)

In [35]:
for i in range(len(liste_final)):
    for j in liste_final[i]:
        name , gene = split_name_gene(j)
        df_final[name][i] = gene
    
df_final.to_csv("../BRH.csv") #path to the BRH.csv
df_final.loc[0]['Acartia-tonsa_adults_SRR6038073_Trinity']

'TRINITY_DN8051_c3_g7::TRINITY_DN8051_c3_g7_i1::g.12616::m.12616'

In [36]:
df_final

,Acartia-tonsa_adults_SRR6038073_Trinity,Amphibalanus-amphitrite_larva_outGroup_SRR1525781_Trinity,Calanoides-carinatus_active-pop_BL_cdhit,Calanus-hyperboreus_female_AF_cdhit,Eucalanus-hyalinus_high-oxygen_CA_cdhit,Eucyclops-serrulatus__SRR1050725_Trinity,Eurytemora-affinis_adults_freshWaters_SRR2551254_Trinity,Labidocera-madurae_femalesPooled_SRR5922844_Trinity,Lernaea-cyprinacea_hote_SRR1104127_Trinity,Lsalmonis_female_SRR7749760_Trinity,...,Onana_female_ind17_Trinity,Osimilis_female_ind18_Trinity,Paracyclopina-nana_SRR1664735_Trinity,Penaeus-aztecus_10Hepatopancreas_outGroup_SRR3711052_Trinity,Pleuromamma-robusta__BU_cdhit,Pseudocalanus-sp_female_BE_cdhit,Pseudodiaptomus-annandalei_1ind_SRR4436675_Trinity,Tigriopus-japonicus_SRR1784985_Trinity,Tigripus-kingsejongensis_SRR2034718_Trinity,Tracheliastes-polycolpus_female_leuciscus_SRR7411027_Trinity
0,TRINITY_DN8051_c3_g7::TRINITY_DN8051_c3_g7_i1:...,TRINITY_DN61102_c1_g1::TRINITY_DN61102_c1_g1_i...,TRINITY_DN11828_c0_g1_i1.p1,TRINITY_DN63972_c0_g1_i2.p1,TRINITY_DN35851_c0_g1_i1.p1,TRINITY_DN97692_c0_g1::TRINITY_DN97692_c0_g1_i...,TRINITY_DN13786_c5_g3::TRINITY_DN13786_c5_g3_i...,TRINITY_DN18851_c1_g2::TRINITY_DN18851_c1_g2_i...,TRINITY_DN68327_c0_g1::TRINITY_DN68327_c0_g1_i...,TRINITY_DN8823_c0_g2::TRINITY_DN8823_c0_g2_i2:...,...,TRINITY_DN9679_c2_g1::TRINITY_DN9679_c2_g1_i1:...,TRINITY_DN7591_c2_g1::TRINITY_DN7591_c2_g1_i1:...,TRINITY_DN17414_c0_g1::TRINITY_DN17414_c0_g1_i...,TRINITY_DN105075_c0_g1::TRINITY_DN105075_c0_g1...,TRINITY_DN51317_c0_g1_i1.p1,TRINITY_DN2670_c0_g1_i1.p2,TRINITY_DN14512_c1_g1::TRINITY_DN14512_c1_g1_i...,TRINITY_DN17552_c4_g5::TRINITY_DN17552_c4_g5_i...,TRINITY_DN12661_c2_g2::TRINITY_DN12661_c2_g2_i...,TRINITY_DN5413_c2_g1::TRINITY_DN5413_c2_g1_i1:...
1,TRINITY_DN7514_c0_g4::TRINITY_DN7514_c0_g4_i1:...,TRINITY_DN59325_c1_g1::TRINITY_DN59325_c1_g1_i...,TRINITY_DN30189_c0_g1_i1.p1,TRINITY_DN46850_c0_g1_i1.p1,TRINITY_DN851_c0_g1_i2.p1,TRINITY_DN72593_c0_g1::TRINITY_DN72593_c0_g1_i...,TRINITY_DN10547_c0_g1::TRINITY_DN10547_c0_g1_i...,TRINITY_DN20017_c0_g2::TRINITY_DN20017_c0_g2_i...,TRINITY_DN42510_c0_g1::TRINITY_DN42510_c0_g1_i...,TRINITY_DN4994_c0_g1::TRINITY_DN4994_c0_g1_i1:...,...,TRINITY_DN10776_c0_g1::TRINITY_DN10776_c0_g1_i...,TRINITY_DN10808_c0_g3::TRINITY_DN10808_c0_g3_i...,TRINITY_DN8016_c0_g1::TRINITY_DN8016_c0_g1_i2:...,TRINITY_DN108631_c0_g1::TRINITY_DN108631_c0_g1...,TRINITY_DN550_c0_g1_i1.p1,TRINITY_DN8910_c0_g1_i2.p2,TRINITY_DN35308_c0_g3::TRINITY_DN35308_c0_g3_i...,TRINITY_DN18524_c0_g1::TRINITY_DN18524_c0_g1_i...,TRINITY_DN3642_c0_g1::TRINITY_DN3642_c0_g1_i1:...,TRINITY_DN2423_c0_g1::TRINITY_DN2423_c0_g1_i2:...
2,TRINITY_DN10685_c1_g19::TRINITY_DN10685_c1_g19...,TRINITY_DN53634_c0_g2::TRINITY_DN53634_c0_g2_i...,TRINITY_DN6312_c0_g1_i3.p2,TRINITY_DN48_c0_g2_i1.p1,TRINITY_DN45421_c0_g1_i1.p1,TRINITY_DN51446_c0_g1::TRINITY_DN51446_c0_g1_i...,TRINITY_DN14267_c3_g9::TRINITY_DN14267_c3_g9_i...,TRINITY_DN16811_c0_g3::TRINITY_DN16811_c0_g3_i...,TRINITY_DN50281_c1_g1::TRINITY_DN50281_c1_g1_i...,TRINITY_DN9546_c0_g1::TRINITY_DN9546_c0_g1_i2:...,...,TRINITY_DN717_c0_g1::TRINITY_DN717_c0_g1_i2::g...,TRINITY_DN845_c0_g1::TRINITY_DN845_c0_g1_i2::g...,TRINITY_DN21515_c0_g1::TRINITY_DN21515_c0_g1_i...,TRINITY_DN109961_c0_g1::TRINITY_DN109961_c0_g1...,TRINITY_DN23_c0_g1_i3.p1,TRINITY_DN263_c0_g2_i1.p1,TRINITY_DN92295_c0_g1::TRINITY_DN92295_c0_g1_i...,TRINITY_DN12875_c0_g1::TRINITY_DN12875_c0_g1_i...,TRINITY_DN11296_c0_g1::TRINITY_DN11296_c0_g1_i...,TRINITY_DN4418_c0_g1::TRINITY_DN4418_c0_g1_i2:...
3,TRINITY_DN10676_c3_g2::TRINITY_DN10676_c3_g2_i...,TRINITY_DN65658_c4_g1::TRINITY_DN65658_c4_g1_i...,TRINITY_DN9141_c0_g1_i1.p1,TRINITY_DN12487_c0_g1_i6.p1,TRINITY_DN12014_c0_g1_i1.p1,TRINITY_DN101055_c1_g1::TRINITY_DN101055_c1_g1...,TRINITY_DN15452_c0_g2::TRINITY_DN15452_c0_g2_i...,TRINITY_DN20581_c0_g4::TRINITY_DN20581_c0_g4_i...,TRINITY_DN11550_c0_g1::TRINITY_DN11550_c0_g1_i...,TRINITY_DN4927_c0_g1::TRINITY_DN4927_c0_g1_i1:...,...,TRINITY_DN10129_c1_g2::TR

# Creating fasta from proteomes

In [48]:
liste_proteomes=[]
for i in os.listdir('/Users/dorvalloic/Documents/BIM/Stage/Best_Reciprocal_Hit/V1/0_proteome_final'):
    liste_proteomes.append(i)
liste_proteomes = sorted(liste_proteomes)


Long computation time to write all file (~30min for 176 files) and the 0_proteome_final/ is the directory with all the proteomes

In [99]:
for index in df_final.index:
    file_R_name= '/Users/dorvalloic/Documents/BIM/Stage/Best_Reciprocal_Hit/V1/0_fasta_BRH/fasta_BRH_'+str(index)
    file_R = open(file_R_name,'w')
    for colname in df_final.columns: 
        proteome_name = '/Users/dorvalloic/Documents/BIM/Stage/Best_Reciprocal_Hit/V1/0_proteome_final/' 
        proteome_name += colname_to_proteome(colname)
        file_P = open(proteome_name,'r')
        GO = False
        for line in file_P:
            if line[0] == '>':
                GO = False

                if line.split(" ")[0][1:] == df_final[colname][index]:
                    GO =True
                    name = make_ID(colname,df_final[colname][index])
                    file_R.write('>'+name+'\n')
            
            if GO == True and line[0] != '>':
                file_R.write(line)
        file_P.close()
    file_R.close()
    print('Fasta ',index,'/',len(df_final.index),'.')

Fasta  0 / 176 .
Fasta  1 / 176 .
Fasta  2 / 176 .
Fasta  3 / 176 .
Fasta  4 / 176 .
Fasta  5 / 176 .
Fasta  6 / 176 .
Fasta  7 / 176 .
Fasta  8 / 176 .
Fasta  9 / 176 .
Fasta  10 / 176 .
Fasta  11 / 176 .
Fasta  12 / 176 .
Fasta  13 / 176 .
Fasta  14 / 176 .
Fasta  15 / 176 .
Fasta  16 / 176 .
Fasta  17 / 176 .
Fasta  18 / 176 .
Fasta  19 / 176 .
Fasta  20 / 176 .
Fasta  21 / 176 .
Fasta  22 / 176 .
Fasta  23 / 176 .
Fasta  24 / 176 .
Fasta  25 / 176 .
Fasta  26 / 176 .
Fasta  27 / 176 .
Fasta  28 / 176 .
Fasta  29 / 176 .
Fasta  30 / 176 .
Fasta  31 / 176 .
Fasta  32 / 176 .
Fasta  33 / 176 .
Fasta  34 / 176 .
Fasta  35 / 176 .
Fasta  36 / 176 .
Fasta  37 / 176 .
Fasta  38 / 176 .
Fasta  39 / 176 .
Fasta  40 / 176 .
Fasta  41 / 176 .
Fasta  42 / 176 .
Fasta  43 / 176 .
Fasta  44 / 176 .
Fasta  45 / 176 .
Fasta  46 / 176 .
Fasta  47 / 176 .
Fasta  48 / 176 .
Fasta  49 / 176 .
Fasta  50 / 176 .
Fasta  51 / 176 .
Fasta  52 / 176 .
Fasta  53 / 176 .
Fasta  54 / 176 .
Fasta  55 / 176 .
Fa

# Concatenate

I have aligned all the seq using muscle, 
sorted all file with : <br/>
 ``` cat ${i} | FastaToTbl - | sort -k 1 | awk '{print ">"$1"\n"$2}' > NameFile ```

and concatenate them with : <br/>
 ``` cut -f1 -d "_" *.fasta|awk -v RS=">" -v FS="\n" -v OFS="\n" '{for(i=2; i<=NF; i++) {seq[$1] = seq[$1]$i}}; END {for(id in seq){print ">"id, seq[id]}}' > combined.fa ```

then change the format from fasta to phylip (maybe passed it througt mega to get a cleaner fasta format)

In [1]:
records = SeqIO.parse("0_muscle/combined_test2.fas", "fasta")
count = SeqIO.write(records, "0_muscle/combined_test2.phylip", "phylip")
print("Converted %i records" % count)

Converted 23 records
